In [1]:
%load_ext autoreload
%autoreload 2

%config IPython.sys.argv=['-Xfrozen_modules=off']

from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

In [4]:
xmon_options = dict(
    connection_pads=dict(
        c=dict(connector_location='90', connector_type='0', 
                # claw_length='50um',
                ground_spacing='9.1um', claw_gap='5.1um', 
                claw_width='19.875um',
                claw_cpw_length='0'
                ),
    ),
    cross_width='30.9um',
    # cross_length=cross_length,
    cross_gap='29.1um',
    # orientation=orientation,
    # pos_x=pos_x,
    # pos_y=pos_y,
    hfss_inductance= 10E-9,
    # aedt_hfss_inductance=LJ * 10**(-9),
    # style="JJ_Doland",
    # Doland_JJ  = Dict(JJ_width = f'{jj_width}nm',
    #                     JJ_flip  = False),
)

# Create a new Transmon Cross object with name 'Q1'
q = TransmonCross(design, 'Q', options=xmon_options)

gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

In [ ]:
!pip install C:\Users\lfl\metal-library\.

# LOM sim

In [5]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [6]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'


In [7]:
import json
from datetime import datetime

def save_simulation_data_to_json(data):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"simulation_data_{current_time}.json"
    
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, indent=4)

In [ ]:
import pandas as pd

ground_spacing_arr = [3, 4, 5, 7, 9, 15, 22, 30]
cross_len_arr = [50+i*50 for i in range(10)]
claw_len_arr = [10+i*50 for i in range(10)]

cross_len_arr = [156.90192620073285,
 157.8269893621786,
 158.76302520041742,
 159.71023010954002,
 160.66880519863838,
 161.6389564341564,
 162.6208947874293,
 163.61483638763337,
 164.62100268037955,
 165.63962059219463,
 166.67092270114702,
 167.71514741388674]

claw_len_arr = [140.61903639160738,
 141.44809880985622,
 142.28699518792078,
 143.13590153860395,
 143.99499810039862,
 144.86446946506587,
 145.74450470986295,
 146.63529753462035,
 147.5370464038769,
 148.44995469429148,
 149.37423084756182,
 150.3100885290915]

all_cap_matricies = []
all_data = []

for i in range(len(ground_spacing_arr)):
    for j in range(len(cross_len_arr)):
        # for k in range(j+1):
        design.components['Q'].options['connection_pads']['c']['ground_spacing'] = str(ground_spacing_arr[i]) + 'um'
        design.components['Q'].options['cross_length'] = str(cross_len_arr[j]) + 'um'
        design.components['Q'].options['connection_pads']['c']['claw_length'] = str(claw_len_arr[j]) + 'um'
        
        Lj = design.components['Q'].options['aedt_hfss_inductance']
        design.components['Q'].options['hfss_inductance'] = Lj
        design.rebuild()

        c1.sim.run(name = 'sweep_v1.0', components=['Q'],
            open_terminations=[('Q', 'c')])
        cap_df = c1.sim.capacitance_matrix
        cap_df['misc'] = [ground_spacing_arr[i], cross_len_arr[j], claw_len_arr[j]]
        all_cap_matricies.append(cap_df)

        print(f'For Qubit_{ground_spacing_arr[i]}_{cross_len_arr[j]}_{claw_len_arr[j]}, the cap matrix is...')
        print(cap_df)

        data = {
            "design_options": design.components['Q'].options,
            "cross_to_ground": cap_df.loc['cross_Q1']['ground_main_plane'],
            "claw_to_ground": cap_df.loc['c_connector_arm_Q1']['ground_main_plane'],
            "cross_to_claw": cap_df.loc['cross_Q1']['ground_main_plane']
        }
        
        # all_data.append(data)
        save_simulation_data_to_json(data)

final_df = pd.concat(all_cap_matricies)